In [1]:
import pandas as pd
from ranker.ranker import Ranker
from ranker.ranker_factory import RankerFactory
from database.market import Market
from database.sec import SEC
from database.adatabase import ADatabase
from processor.processor import Processor as p
from datetime import datetime, timedelta
from tqdm import tqdm
import numpy as np
from modeler_strats.universal_modeler import UniversalModeler

In [2]:
## initializing ranker_class
ranker_class = RankerFactory.build(Ranker.WEEKLY_STOCK_ROLLING_RANKER)

In [3]:
backtest_start_date = datetime(2016,1,1)
backtest_end_date = datetime(2023,1,1)

In [4]:
## creating database objects
market = Market()
sec = SEC()
modeler_strat = UniversalModeler()

In [5]:
market.connect()
sp500 = market.retrieve("sp500")
market.disconnect()
sp500 = sp500.rename(columns={"Symbol":"ticker"})

In [6]:
market.connect()
sec.connect()
training_sets = []
for ticker in tqdm(sp500["ticker"].unique()):
    try:
        prices = market.retrieve_ticker_prices(ranker_class.asset_class.value,ticker)
        prices = p.column_date_processing(prices)
        ticker_data = ranker_class.training_set(ticker,prices)
        training_sets.append(ticker_data)
    except Exception as e:
        print(str(e))
        continue
market.disconnect()
sec.disconnect()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 490/490 [00:53<00:00,  9.13it/s]


In [7]:
data = pd.concat(training_sets)

In [12]:
training_data = data.dropna().copy().sort_values(["year","week"])

In [14]:
training_data = training_data.groupby(["year","quarter","week","ticker"]).mean().reset_index()

In [15]:
ranker_class.db.connect()
relevant_columns = list(set(list(training_data.columns)) - set(ranker_class.factors))
ranker_class.db.store("sim",training_data[relevant_columns])
ranker_class.db.disconnect()